In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2

In [2]:
dir(psycopg2)

['BINARY',
 'Binary',
 'DATETIME',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NUMBER',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'ROWID',
 'STRING',
 'Time',
 'TimeFromTicks',
 'Timestamp',
 'TimestampFromTicks',
 'Warning',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__libpq_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_connect',
 '_ext',
 '_json',
 '_psycopg',
 '_range',
 'apilevel',
 'compat',
 'connect',
 'errors',
 'extensions',
 'paramstyle',
 'threadsafety',
 'tz']

In [3]:
## Set up steps

dbname = 'qdesulho'
user = 'qdesulho'
password = 'JGeDV6muogRvBZvrAfYst69Yeb9mQlBg'
host = 'rajje.db.elephantsql.com'

In [13]:
pg_conn = psycopg2.connect(dbname=dbname, user=user, 
                           password=password, host = host)

In [14]:
## check to see if we have a connection object
pg_conn

<connection object at 0x106dd1b50; dsn: 'user=qdesulho password=xxx dbname=qdesulho host=rajje.db.elephantsql.com', closed: 0>

In [15]:
## make a cursor
pg_curs = pg_conn.cursor()

In [16]:
## structure of the table you want to create
create_table_statement = """
CREATE TABLE test_table (
  id        SERIAL PRIMARY KEY,
  name  varchar(40) NOT NULL,
  data    JSONB
);
"""

## code to create the table you wanted
pg_curs.execute(create_table_statement)

## commiting the changes
pg_conn.commit()

In [17]:
## data for the table you created

insert_statement = """
INSERT INTO test_table (name, data) VALUES
(
  'A row name',
  null
),
(
  'Another row, with JSON',
  '{ "a": 1, "b": ["dog", "cat", 42], "c": true }'::JSONB
);
"""

## puts the data into the table
pg_curs.execute(insert_statement)

## commits the changes you made
pg_conn.commit()


In [18]:
## SQL search
query = 'SELECT * FROM "public"."test_table" LIMIT 100'

## do the search
pg_curs.execute(query)

##s show the results
pg_curs.fetchall()

[(1, 'A row name', None),
 (2, 'Another row, with JSON', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

## RPG Data

In [22]:
import sqlite3

In [26]:
sl_conn = sqlite3.connect('rpg_db.sqlite3')

In [27]:
sl_curs = s1_conn.cursor()

In [28]:
row_count = 'SELECT COUNT(*) FROM charactercreator_character'
sl_curs.execute(row_count).fetchall()

[(302,)]

In [31]:
## we want to move characters from sqlite3 to postgres
## select the characters
get_characters = '''SELECT * FROM charactercreator_character'''

## execute search and save results
characters = sl_curs.execute(get_characters).fetchall()

In [33]:
## examine first 5 results
characters[:5]

[(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1),
 (2, 'Optio dolorem ex a', 0, 0, 10, 1, 1, 1, 1),
 (3, 'Minus c', 0, 0, 10, 1, 1, 1, 1),
 (4, 'Sit ut repr', 0, 0, 10, 1, 1, 1, 1),
 (5, 'At id recusandae expl', 0, 0, 10, 1, 1, 1, 1)]

In [34]:
## count how many results
len(characters)

302

In [35]:
## since we are moving from sqlite3 to postgreSQL we need to see how data was structured
sl_curs.execute('PRAGMA table_info(charactercreator_character);').fetchall()

[(0, 'character_id', 'integer', 1, None, 1),
 (1, 'name', 'varchar(30)', 1, None, 0),
 (2, 'level', 'integer', 1, None, 0),
 (3, 'exp', 'integer', 1, None, 0),
 (4, 'hp', 'integer', 1, None, 0),
 (5, 'strength', 'integer', 1, None, 0),
 (6, 'intelligence', 'integer', 1, None, 0),
 (7, 'dexterity', 'integer', 1, None, 0),
 (8, 'wisdom', 'integer', 1, None, 0)]

In [36]:
## we now need to create a table to put this information into
create_character_table = '''
CREATE TABLE charactercreator_character (
    character_id SERIAL PRIMARY KEY,
    name VARCHAR(30),
    level INT,
    exp INT,
    hp INT,
    strength INT,
    intelligence INT,
    dexterity INT,
    wisdom INT
);
'''

In [38]:
## I got an error so I re-established the connection
pg_conn = psycopg2.connect(dbname=dbname, user=user, 
                           password=password, host = host)
## make a cursor
pg_curs = pg_conn.cursor()

In [39]:
## create the table using the pg_cursor
pg_curs.execute(create_character_table)

## save changes
pg_conn.commit()

In [40]:
## look at tables
show_tables = """
SELECT
   *
FROM
   pg_catalog.pg_tables
WHERE
   schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
"""
pg_curs.execute(show_tables)
pg_curs.fetchall()

[('public', 'test_table', 'qdesulho', None, True, False, False, False),
 ('public',
  'charactercreator_character',
  'qdesulho',
  None,
  True,
  False,
  False,
  False)]

In [46]:
## in case you make a mistake you can delete all the rows
delete_table_rows = '''DELETE FROM charactercreator_character
'''
pg_curs.execute(delete_table_rows)
pg_conn.commit()

In [47]:
## execute and commit deletion of rows
pg_curs.execute('SELECT * FROM charactercreator_character')
pg_curs.fetchall()

[]

In [48]:
for character in characters:
    insert_character = '''
    INSERT INTO charactercreator_character
    (name, level, exp, hp, strength, intelligence, dexterity, wisdom)
    VALUES ''' + str(character[1:]) + ";"
    
    pg_curs.execute(insert_character)
    pg_conn.commit()

In [50]:
pg_curs.execute('SELECT * FROM charactercreator_character')
len(pg_curs.fetchall())

302

## TITANIC

### Create new tables for database

In [51]:
## create a table for passengers
passengers_table = '''
CREATE TABLE passengers (
id SERIAL PRIMARY KEY,
name TEXT,  
sex TEXT, 
age FLOAT,
sibsp INT,
parch INT
)
'''
pg_curs.execute(passengers_table)
pg_conn.commit()

In [52]:
## create a survival table
survival_table = '''
CREATE TABLE survival (
id SERIAL PRIMARY KEY,
survived INT
)
'''
pg_curs.execute(survival_table)
pg_conn.commit()

In [54]:
## create a fare info table
fare_info = '''
CREATE TABLE fare_info (
id SERIAL PRIMARY KEY,
pclass INT,
ticket TEXT,
fare FLOAT,
cabin TEXT,
embarked TEXT
)
'''
pg_curs.execute(fare_info)
pg_conn.commit()

### Move data from CSV to Database

In [59]:
## re-establish connection
pg_conn = psycopg2.connect(dbname=dbname, user=user, 
                           password=password, host = host)
## make a cursor
pg_curs = pg_conn.cursor()

In [60]:
with open('passengers.csv', 'r') as row:
    next(row)# Skip the header row.
    pg_curs.copy_from(row, 'passengers', sep=',')
    
with open('survival.csv', 'r') as row:
    next(row) 
    pg_curs.copy_from(row, 'survival', sep=',')

with open('trip_info.csv', 'r') as row:
    next(row)
    pg_curs.copy_from(row, 'fare_info', sep=',')
    
pg_conn.commit()    

## Run a query to show that it works

In [64]:
query = '''
SELECT * FROM "public"."fare_info" LIMIT 100
'''
pg_curs.execute(query)
pg_curs.fetchall()

[(1, 3, 'A/5 21171', 7.25, '?', 'S'),
 (2, 1, 'PC 17599', 71.2833, 'C85', 'C'),
 (3, 3, 'STON/O2. 3101282', 7.925, '?', 'S'),
 (4, 1, '113803', 53.1, 'C123', 'S'),
 (5, 3, '373450', 8.05, '?', 'S'),
 (6, 3, '330877', 8.4583, '?', 'Q'),
 (7, 1, '17463', 51.8625, 'E46', 'S'),
 (8, 3, '349909', 21.075, '?', 'S'),
 (9, 3, '347742', 11.1333, '?', 'S'),
 (10, 2, '237736', 30.0708, '?', 'C'),
 (11, 3, 'PP 9549', 16.7, 'G6', 'S'),
 (12, 1, '113783', 26.55, 'C103', 'S'),
 (13, 3, 'A/5. 2151', 8.05, '?', 'S'),
 (14, 3, '347082', 31.275, '?', 'S'),
 (15, 3, '350406', 7.8542, '?', 'S'),
 (16, 2, '248706', 16.0, '?', 'S'),
 (17, 3, '382652', 29.125, '?', 'Q'),
 (18, 2, '244373', 13.0, '?', 'S'),
 (19, 3, '345763', 18.0, '?', 'S'),
 (20, 3, '2649', 7.225, '?', 'C'),
 (21, 2, '239865', 26.0, '?', 'S'),
 (22, 2, '248698', 13.0, 'D56', 'S'),
 (23, 3, '330923', 8.0292, '?', 'Q'),
 (24, 1, '113788', 35.5, 'A6', 'S'),
 (25, 3, '349909', 21.075, '?', 'S'),
 (26, 3, '347077', 31.3875, '?', 'S'),
 (27, 3, '2